In [4]:
import pandas as pd 
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text

### Link campaign to creator 

In [5]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)

## Influenceur 

Rejoint une campaign  ---> créer une participation => génération d'un lien de partenariat et/ou un code promo 

In [7]:
create_table_participation = text('''CREATE TABLE IF NOT EXISTS campaign_participation (
    participation_id INTEGER PRIMARY KEY AUTOINCREMENT,
    participation_link TEXT,
    participation_code  TEXT,                           
    campaign_id INTEGER NOT NULL,
    creator_id INTEGER NOT NULL,
    status VARCHAR(100),
    role VARCHAR(100),
    join_date DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (campaign_id) REFERENCES campaigns(campaign_id) ON DELETE CASCADE,
    FOREIGN KEY (creator_id) REFERENCES creators(creator_id) ON DELETE CASCADE,
    UNIQUE (campaign_id, creator_id)  -- ensures each pair is unique
);''')

In [8]:
# Assuming 'engine' is already defined and imported from SQLAlchemy setup
with engine.connect() as connection:
    connection.execute(create_table_participation)
    print("Participations table created successfully.")

2024-04-17 15:12:02,341 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 15:12:02,341 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS campaign_participation (
    participation_id INTEGER PRIMARY KEY AUTOINCREMENT,
    campaign_id INTEGER NOT NULL,
    creator_id INTEGER NOT NULL,
    status VARCHAR(100),
    role VARCHAR(100),
    join_date DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (campaign_id) REFERENCES campaigns(campaign_id) ON DELETE CASCADE,
    FOREIGN KEY (creator_id) REFERENCES creators(creator_id) ON DELETE CASCADE,
    UNIQUE (campaign_id, creator_id)  -- ensures each pair is unique
);
2024-04-17 15:12:02,341 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ()
Participations table created successfully.
2024-04-17 15:12:02,349 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
def add_participation(campaign_id, creator_id, **kwargs):
    data = {'campaign_id': campaign_id, 'creator_id': creator_id, **kwargs}
    columns = ', '.join(data.keys())
    placeholders = ', '.join(f":{key}" for key in data.keys())

    insert_query = text(f"""
        INSERT INTO campaign_participation ({columns})
        VALUES ({placeholders});
    """)
    with engine.begin() as connection:
        connection.execute(insert_query, data)
        print(f"Participation added: Campaign ID {campaign_id}, Creator ID {creator_id}, Details: {kwargs}")


def modify_participation(participation_id, **kwargs):
    updates = ', '.join(f"{key} = :{key}" for key in kwargs)
    update_query = text(f"""
        UPDATE campaign_participation
        SET {updates}
        WHERE participation_id = :participation_id;
    """)
    kwargs['participation_id'] = participation_id  # Include for WHERE clause
    with engine.begin() as connection:
        result = connection.execute(update_query, kwargs)
        if result.rowcount > 0:
            print(f"Participation ID {participation_id} updated successfully.")
        else:
            print(f"No participation found with ID {participation_id}.")



def get_participation(**kwargs):
    if not kwargs:
        print("No parameters provided to search for participation.")
        return None

    # Constructing the WHERE clause based on kwargs
    conditions = [f"{key} = :{key}" for key in kwargs]
    where_clause = " AND ".join(conditions)
    select_query = text(f"SELECT * FROM campaign_participation WHERE {where_clause}")

    print(select_query)  # Optional: Print the query for debugging
    with engine.connect() as connection:
        result = connection.execute(select_query, kwargs).fetchone()
        print(result)  # Optional: Print the raw result for debugging
        
        if result:
            # Assuming we know the columns in order of the table
            participation_dict = {
                'participation_id': result[0],
                'campaign_id': result[1],
                'creator_id': result[2],
                'status': result[3],
                'role': result[4],
                'join_date': result[5]
            }
            
            return participation_dict
        else:
            print("Participation not found.")
            return None

In [19]:
pd.read_sql('SELECT * FROM campaign_participation',engine)

2024-04-17 15:17:11,578 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 15:17:11,579 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM campaign_participation")
2024-04-17 15:17:11,579 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 15:17:11,580 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM campaign_participation")
2024-04-17 15:17:11,580 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 15:17:11,581 INFO sqlalchemy.engine.Engine SELECT * FROM campaign_participation
2024-04-17 15:17:11,581 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 15:17:11,583 INFO sqlalchemy.engine.Engine ROLLBACK


,participation_id,campaign_id,creator_id,status,role,join_date
0,1,1,5,None,None,2024-04-17 13:14:55
1,2,1,7,Active,Influenceur,2024-04-17 13:17:09


In [18]:
# Adding a participation with minimal required information
add_participation(campaign_id=1, creator_id=7, role='Influenceur', status='Active')

2024-04-17 15:17:09,648 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 15:17:09,649 INFO sqlalchemy.engine.Engine 
        INSERT INTO campaign_participation (campaign_id, creator_id, status, role)
        VALUES (?, ?, ?, ?);
    
2024-04-17 15:17:09,649 INFO sqlalchemy.engine.Engine [cached since 133.7s ago] (1, 7, 'Active', 'Influenceur')
Participation added: Campaign ID 1, Creator ID 7.
2024-04-17 15:17:09,652 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
get_participation(campaign_id=1)

SELECT * FROM campaign_participation WHERE campaign_id = :campaign_id
2024-04-17 15:17:46,362 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 15:17:46,363 INFO sqlalchemy.engine.Engine SELECT * FROM campaign_participation WHERE campaign_id = ?
2024-04-17 15:17:46,363 INFO sqlalchemy.engine.Engine [generated in 0.00094s] (1,)
(1, 1, 5, None, None, '2024-04-17 13:14:55')
2024-04-17 15:17:46,364 INFO sqlalchemy.engine.Engine ROLLBACK


{'participation_id': 1,
 'campaign_id': 1,
 'creator_id': 5,
 'status': None,
 'role': None,
 'join_date': '2024-04-17 13:14:55'}